# Module 1 - Extracting Gaussian features from input data
*Updated on 2025-06-25*

This Module is divided into five steps, taking input data, normalizing it, extracting Gaussian features from input data, reconstructing data by using Gaussian features and finally, determine the separation point of the two isomers.

Users will be asked to navigate to the directory where the input data files are stored. Output excel files and plots will be stored in this same folder.

The output excel files are: **CombinedAnalysis.xlsx, meanAllsigma.xlsx, Intersected gauss.xlsx** .
The output plots are stored in folders: **RawPlots, NormalizedPlots, Gal_GaussianPlots, Glc_GaussianPlots, GaussianIntersectPlots** .

## INSTALLATION OF IMPORTANT LIBRARIES AND PACKAGES

In [ ]:
%matplotlib inline

# Import basic system parameters and functions
import sys
import subprocess
import pkg_resources

# Several packages needed to be installed. We first will check to see if your environment has already had these packages installed.
required = {'pandas', 'seaborn', 'numpy', 'csaps', 'tensorflow', 'openpyxl', 'lmfit', 'tqdm'}
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = required - installed

if missing:
    python = sys.executable
    subprocess.check_call([python, '-m', 'pip', 'install', *missing], stdout = subprocess.DEVNULL)

# After checking, we now will import from the installed packages
import os
import pandas as pd
import seaborn as sns
import pylab as plb
import matplotlib.pyplot as mpl
from matplotlib import pyplot as plt
from matplotlib import mlab
import numpy as np
import heapq
import csaps
import shutil
import time
import tkinter
from tkinter import filedialog
from tkinter import *
from csaps import csaps
from random import seed
from random import choice
from numpy import asarray as ar
from numpy import exp, linspace, random, pi, sqrt
from scipy import stats
from scipy import interpolate
from scipy.optimize import curve_fit
from scipy.stats import linregress
from scipy.stats import norm
from tqdm import tqdm
import math
import tensorflow as tn
from tensorflow.keras import layers
from tensorflow.keras import models
from lmfit import Model, conf_interval, report_ci
from lmfit.models import ExpressionModel

# Set Directory Location
currentpath = os.getcwd()
os.chdir(currentpath)

# Set up folder for outputs and plots

M1_RawPlots = './Module1_Outputs/Plots/RawPlots'
M1_NormPlots = './Module1_Outputs/Plots/NormalizedPlots'
M1_Gal_GaussPlots = './Module1_Outputs/Plots/Gal_GaussianPlots'
M1_Glc_GaussPlots = './Module1_Outputs/Plots/Glc_GaussianPlots'
M1_Intersect_GaussPlots = './Module1_Outputs/Plots/GaussianIntersectPlots'


if not os.path.exists(M1_RawPlots):
    os.makedirs(os.path.join('Module1_Outputs', 'Plots', 'RawPlots'))
    os.chmod(M1_RawPlots,0o666)
else:
    os.chmod(M1_RawPlots,0o666)
    shutil.rmtree(M1_RawPlots)
    os.makedirs(os.path.join('Module1_Outputs', 'Plots', 'RawPlots'))
    
if not os.path.exists(M1_NormPlots):
    os.makedirs(os.path.join('Module1_Outputs', 'Plots', 'NormalizedPlots'))
    os.chmod(M1_NormPlots,0o666)
else:
    os.chmod(M1_NormPlots,0o666)
    shutil.rmtree(M1_NormPlots)
    os.makedirs(os.path.join('Module1_Outputs', 'Plots', 'NormalizedPlots'))

if not os.path.exists(M1_Gal_GaussPlots):
    os.makedirs(os.path.join('Module1_Outputs', 'Plots', 'Gal_GaussianPlots'))
    os.chmod(M1_Gal_GaussPlots,0o666)
else:
    os.chmod(M1_Gal_GaussPlots,0o666)
    shutil.rmtree(M1_Gal_GaussPlots)
    os.makedirs(os.path.join('Module1_Outputs', 'Plots', 'Gal_GaussianPlots'))

if not os.path.exists(M1_Glc_GaussPlots):
    os.makedirs(os.path.join('Module1_Outputs', 'Plots', 'Glc_GaussianPlots'))
    os.chmod(M1_Glc_GaussPlots,0o666)
else:
    os.chmod(M1_Glc_GaussPlots,0o666)
    shutil.rmtree(M1_Glc_GaussPlots)
    os.makedirs(os.path.join('Module1_Outputs', 'Plots', 'Glc_GaussianPlots'))

if not os.path.exists(M1_Intersect_GaussPlots):
    os.makedirs(os.path.join('Module1_Outputs', 'Plots', 'GaussianIntersectPlots'))
    os.chmod(M1_Intersect_GaussPlots,0o666)
else:
    os.chmod(M1_Intersect_GaussPlots,0o666)
    shutil.rmtree(M1_Intersect_GaussPlots)
    os.makedirs(os.path.join('Module1_Outputs', 'Plots', 'GaussianIntersectPlots'))


## STEP 1. Obtain empirical data

In [ ]:
# Browse to file location
root = tkinter.Tk()
InputDir = filedialog.askdirectory(parent= root,
                                    initialdir="/",
                                    title = "Browse to where files are stored")
if len(InputDir) > 0:
    print ("You chose %s" % InputDir)

root.withdraw() #use to hide tkinter window ; this line is important to prevent tkinter from hanging
root.update()

# Select the files for each isomer.
## Files are read and stored as DataFrame.
InputFile1 = filedialog.askopenfile(parent=root,
                                   mode = 'rb',
                                   title = "Upload training data for Glucosyl Ceramide stereoisomers.")
if InputFile1 != None:
    Glcsingle = pd.read_excel(InputFile1)

root.withdraw()

InputFile2 = filedialog.askopenfile(parent=root,
                                   mode = 'rb',
                                   title = "Upload training data for Galactosyl Ceramide stereoisomers.")
if InputFile2 != None:
    Galsingle = pd.read_excel(InputFile2)
    
root.withdraw()
root.update()

# Reformat the above files to round CoV to 1 decimals
Galsingle['COV']=Galsingle['COV'].round(1)
Glcsingle['COV']=Glcsingle['COV'].round(1)

## STEP 2. Checking empirical data and extracting important parameters

In [ ]:
# Note: SV range may not be the same for all isomers because a well-resolved peak may not be produced at all SV, which then would be excluded in QC step.

# i) The number of unique lipid species in the input files must equal one another.
nGlc = len(pd.unique(Glcsingle['Lipid Species']))
Glc_unique = set(Glcsingle['Lipid Species'])
print("Number of unique Glc species are", nGlc, "; and they are ", Glc_unique)

nGal = len(pd.unique(Galsingle['Lipid Species']))
Gal_unique = set(Galsingle['Lipid Species'])
print("Number of unique Gal species are", nGal, "; and they are ", Gal_unique)

# In this step, the code will check that the number of unique Glc species does equal the number of Gal species.
# If they equal each other, code will continue.
# If not, code will stop. User is asked to check their input dataset before retrying the code.
if (nGlc == nGal):
    nLipids = nGlc
    list_Lipids = Glc_unique
    print("The number of unique Glc isomer equals the number of unique Gal isomer. We will proceed!")
else:
    print("The number of Glc lipid species DOES NOT MATCH the number of Gal lipid species. Algorithm cannot be built!")
    print(input("We will now exit the code. Please check your input datafiles, and restart the code with the correct input files. Type 'EXIT' "))
    exit()
    
# ii. Determine the number of replicates in dataset. The number of replicates in both isomers datafiles must match.
if ((Glcsingle['replicateNum'].max()) == Galsingle['replicateNum'].max()):
    nRep = Glcsingle['replicateNum'].max()
    print("The number of experimental replicates for each Glc and for each Gal isomer is", nRep)
    print("Since they equal each other, we will proceed!")
    
else:
    print("The number of experiment replicates for Glc lipid species DOES NOT MATCH that for Gal lipid species. Algorithm cannot be built!")
    print(input("We will now exit the code. Please check your input datafiles, and restart the code with the correct input files. Type 'EXIT' "))
    exit()

# iii. Determine the CoV range used in input dataset. This COV range must be the same for every species, isomer, at all measured SV.
if( (float(Glcsingle['COV'].min()) == float(Galsingle['COV'].min())) and (float(Glcsingle['COV'].max()) == float(Galsingle['COV'].max())) ):
    minCoV = float(Glcsingle['COV'].min())
    maxCoV = float(Galsingle['COV'].max())
    stepCoV = float((maxCoV - minCoV) / ((Glcsingle['COV'].nunique())-1))
    CoVList = pd.unique(Glcsingle['COV'])
    print("The range of COV in this dataset is from ", minCoV, " to ", maxCoV, " with step of ", stepCoV, " V for both isomers." )
    print("Since the range matches, we will proceed!")

else:
    print("We have a problem. The CoV range in this experiment is not the same for both isomers.")
    print(input("We will now exit the code. Please check your input datafiles, and restart the code with the correct input files. You may need to perform another optimization experiment with matching CoV range. Type 'EXIT' "))
    exit()

## STEP 3. Assess the original data, normalize and create ionogram plots of Glc and Gal isomer together

In [ ]:
# i. A dictionary is created for all dataframes generarated for each lipid species for each isomer.
dataGlc = {}
for x in Glc_unique:
    dataGlc[x] = pd.DataFrame()
    dataGlc[x] = Glcsingle.loc[Glcsingle['Lipid Species'] == x]

    
dataGal = {}
for x in Gal_unique:
    dataGal[x] = pd.DataFrame()
    dataGal[x] = Galsingle.loc[Galsingle['Lipid Species'] == x]
    
keys_dataGlc = list(dataGlc.keys())
keys_dataGal = list(dataGal.keys())

# ii. Normalize the raw data of each isomer by the maximum intensity at each SV. Then plot the normalized data.
raw_GlcPeakCoV = pd.DataFrame()
raw_GalPeakCoV = pd.DataFrame()
norm_rawGlc = pd.DataFrame()
norm_rawGal = pd.DataFrame()

pd.options.mode.chained_assignment = None

for i in range(len(list_Lipids)):
        Galtemp = Galsingle[Galsingle['Lipid Species']==keys_dataGal[i]]
        Glctemp = Glcsingle[Glcsingle['Lipid Species']==keys_dataGlc[i]]
        titleGal = 'GalCer(d18:1/' + str(keys_dataGal[i]) + ')'
        titleGlc = 'GlcCer(d18:1/' + str(keys_dataGlc[i]) + ')'
    
        minSV = Galtemp['SV'].min()
        maxSV = Galtemp['SV'].max()
        stepSV = int((maxSV - minSV) / ((Galtemp['SV'].nunique())-1))         
        
        for duplicate in range(1, nRep+1):
            for SV in range(minSV, (maxSV + stepSV), stepSV):
                GaltempSV = Galtemp[(Galtemp['SV']==SV)&(Galtemp['replicateNum']==duplicate)]
                GlctempSV = Glctemp[(Glctemp['SV']==SV)&(Glctemp['replicateNum']==duplicate)]
                if (GaltempSV['Intensity'].max()) != 0 and (GaltempSV['Intensity'].max()) > 100 : # Intensity Criteria
                    raw_GaltempPeakCoV = GaltempSV.loc[GaltempSV['Intensity']==GaltempSV['Intensity'].max()]
                    raw_GalPeakCoV = pd.concat([raw_GalPeakCoV, raw_GaltempPeakCoV])
                    raw_GalPeakCoV.reset_index(drop = True, inplace = True)
                    
                if (GlctempSV['Intensity'].max()) != 0 and (GlctempSV['Intensity'].max()) > 100 : # Intensity Criteria
                    raw_GlctempPeakCoV = GlctempSV.loc[GlctempSV['Intensity']==GlctempSV['Intensity'].max()]
                    raw_GlcPeakCoV = pd.concat([raw_GlcPeakCoV, raw_GlctempPeakCoV])
                    raw_GlcPeakCoV.reset_index(drop = True, inplace = True)
                    
                raw_GalPeakCoV.sort_values(by = ['Lipid Species', 'SV'], ascending=[True, True], inplace=True, 
                                           na_position='first', ignore_index=True, key=None)
                
                raw_GlcPeakCoV.sort_values(by = ['Lipid Species', 'SV'], ascending=[True, True], inplace=True,
                                           na_position='first', ignore_index=True, key=None)                
                
                # Plot raw data
                ## Glc first
                plt.figure()
                plt.plot(GlctempSV['COV'],GlctempSV['Intensity'], 'o', color='grey', linewidth=1)
                plt.ylabel('Signal Intensity', fontsize = 12)
                plt.xlabel('CoV', fontsize = 12)
                plt.title((titleGlc +' at '+'SV '+str(SV) + ' ' + 'Rep ' + str(duplicate)), fontsize = 14)
                #plt.legend(["Signal Intensity"])
                temp_Glc = plt.gcf()
                plt.show()
                plt.draw()
                plotname = 'Glc' + keys_dataGlc[i] + '_SV' + str(SV) + '_rep' + str(duplicate)
                plotname = plotname.replace(":","-")
                plotpath = os.path.join(currentpath, M1_RawPlots,plotname)
                temp_Glc.savefig(plotpath)
                
                
                ## Gal
                plt.plot(GaltempSV['COV'],GaltempSV['Intensity'], 'o', color='grey', linewidth=1)
                plt.ylabel('Signal Intensity', fontsize = 12)
                plt.xlabel('CoV', fontsize = 12)
                plt.title((titleGal +' at '+'SV '+str(SV) + ' ' + 'Rep ' + str(duplicate)), fontsize = 14)
                #plt.legend(["Signal Intensity"])
                temp_Gal = plt.gcf()
                plt.show()
                plt.draw()
                plotname = 'Gal' + keys_dataGal[i] + '_SV' + str(SV) + '_rep' + str(duplicate)
                plotname = plotname.replace(":","-")
                plotpath = os.path.join(currentpath, M1_RawPlots,plotname)
                temp_Gal.savefig(plotpath)
               
                          
                #Normalize the raw data
                GaltempSV['NormPeakIntensity'] = GaltempSV["Intensity"]/GaltempSV["Intensity"].max()
                norm_rawGal = pd.concat([norm_rawGal, GaltempSV])
                norm_rawGal.reset_index(drop = True, inplace = True)
                
                GlctempSV['NormPeakIntensity'] = GlctempSV["Intensity"]/GlctempSV["Intensity"].max()
                norm_rawGlc = pd.concat([norm_rawGlc, GlctempSV])
                norm_rawGlc.reset_index(drop = True, inplace = True)
                
                # Plot normalized data
                ## Glc first
                plt.plot(GlctempSV['COV'],GlctempSV['NormPeakIntensity'], 'o', color='grey', linewidth=1)
                plt.plot(GlctempSV['COV'],GlctempSV['NormPeakIntensity'],color='grey',label='GlcCer' )
                plt.ylabel('Normalized  Signal Intensity', fontsize = 12)
                plt.xlabel('CoV', fontsize = 12) 
                plt.title((titleGlc +' at '+'SV '+str(SV) + ' ' + 'Rep ' + str(duplicate)), fontsize = 14)
                plt.legend(["Normalized\nSignal"])
                temp_Glc = plt.gcf()
                plt.show()
                plt.draw()
                plotname = 'Glc' + keys_dataGal[i] + '_SV' + str(SV) + '_rep' + str(duplicate)
                plotname = plotname.replace(":","-")
                plotpath = os.path.join(currentpath, M1_NormPlots, plotname)
                temp_Glc.savefig(plotpath)

                
                ## Gal
                plt.plot(GaltempSV['COV'],GaltempSV['NormPeakIntensity'], 'o', color='grey', linewidth=1)
                plt.plot(GaltempSV['COV'],GaltempSV['NormPeakIntensity'],color='grey',label='GalCer')
                plt.ylabel('Normalized Signal Intensity', fontsize = 12)
                plt.xlabel('CoV', fontsize = 12) 
                plt.title((titleGal +' at '+'SV '+str(SV) + ' ' + 'Rep ' + str(duplicate)), fontsize = 14)
                plt.legend(["Normalized\nSignal"])
                temp_Gal = plt.gcf()
                plt.show()
                plt.draw()
                plotname = 'Gal' + keys_dataGal[i] + '_SV' + str(SV) + '_rep' + str(duplicate)
                plotname = plotname.replace(":","-")
                plotpath = os.path.join(currentpath, M1_NormPlots, plotname)
                temp_Gal.savefig(plotpath)
                
    
plt.close()

# Output raw CoV files for checking
raw_GalPeakCoV.to_excel('Module1_Outputs/raw_GalPeakCoV.xlsx')
raw_GlcPeakCoV.to_excel('Module1_Outputs/raw_GlcPeakCoV.xlsx')

# Output normalized raw datafiles for checking
norm_rawGal.to_excel('Module1_Outputs/norm_rawGal.xlsx')
norm_rawGlc.to_excel('Module1_Outputs/norm_rawGlc.xlsx')


In [ ]:
# Message to users
print("All plots from normalized input data are saved in folder called 'NormalizedPlots.' ")

## STEP 4. Gaussian fit the normalized data, then obtain gaussian features (sigma and mu) from the fitting

In [ ]:
# i. Gaussian fit to normalized data. As well, extract Gaussian features for each replicate at each SV.

norm_GalPeakFeatures = pd.DataFrame()
norm_GlcPeakFeatures = pd.DataFrame()

pd.options.mode.chained_assignment = None
plt.close()
lipid_isomer = ["Gal", "Glc"]

## Define Gaussian function
def gaussian(x, mu, sigma):
   y = 1 * exp(-0.5*((x-mu)/sigma)**2)
   return y

## Looping to derive function and features from data

for isomer in lipid_isomer:
    if isomer == "Gal":
        for i in range(len(list_Lipids)):
            Galtemp = (norm_rawGal[norm_rawGal['Lipid Species']==keys_dataGal[i]]).dropna()        
            Galtempmu = raw_GalPeakCoV[raw_GalPeakCoV['Lipid Species']==keys_dataGal[i]]
            
            minSV_Gal = Galtemp['SV'].min()
            maxSV_Gal = Galtemp['SV'].max()            
            stepSV = int((maxSV_Gal - minSV_Gal) / ((Galtemp['SV'].nunique())-1))
            
            for duplicate in range(1, nRep+1):
                Gal_rep = Galtemp[Galtemp['replicateNum']==duplicate]
                Gal_mu_rep = Galtempmu[Galtempmu['replicateNum']==duplicate]
                
                for SV in range(minSV_Gal, maxSV_Gal+stepSV, stepSV):
                    GaltempSV = Gal_rep[(Gal_rep['SV']==SV)]
                    GaltempmuSV = Gal_mu_rep[(Gal_mu_rep['SV']==SV)]
                    
                    xGaldata = ar(GaltempSV["COV"])
                    yGaldata = ar(GaltempSV["NormPeakIntensity"])
                    mu_Gal = float(sum(GaltempmuSV['COV'])/(GaltempmuSV.shape[0])) #this will be the start point
                    
                    # Use Model fit to fit data. Here we use Lmfit to build a Model wrapping the gaussian model function
                    # We assign initial values for this model with mu as the average mu across all CoV for a specific SV (peakCoV)
                    # and sigma at 1.
                    # As well, here, we use method 'fit' of the model to fit data to this model with the initial parameters
                    gmodel = Model(gaussian)
                    Gal_result = gmodel.fit(yGaldata, x=xGaldata, mu = mu_Gal, sigma = 1)
                    Gal_uncertain = Gal_result.eval_uncertainty(sigma = 3)

                    # Modified parameters set 1 for less optimal fit_FOR GAL
                    modified_Gal_params1 = Gal_result.init_params.copy()
                    modified_Gal_params1['mu'].value = mu_Gal
                    modified_Gal_params1['sigma'].value = 0.75
                    
                    # Calculate new model 1 with modified parameters set 1_FOR GAL
                    y_Gal_modified1 = gmodel.eval(modified_Gal_params1, x=xGaldata)
                    
                    # Modified parameters set 2 for less optimal fit_FOR GAL
                    modified_Gal_params2 = Gal_result.init_params.copy()
                    modified_Gal_params2['mu'].value = mu_Gal
                    modified_Gal_params2['sigma'].value = 0.25
                    
                    # Calculate new model 2 with modified parameters set 2_FOR GAL
                    y_Gal_modified2 = gmodel.eval(modified_Gal_params2, x=xGaldata)
                    
                    # The ModelResult methods provide fit parameters and results from the model. We use 'summary' method
                    # to output the statistics and attributes of the model's results. This is a dictionary.
                    # We then derive the value for key "best_values" from the summary, which are mu and sigma
                    gmodel_report = Gal_result.fit_report()
                    gmodel_summary = Gal_result.summary()
                    bestfit_features = gmodel_summary["best_values"]
                    bestfit_features_list = list(bestfit_features.values())
                    r_squared = gmodel_summary["rsquared"]
                    
                    
                    # The best fit values are now derived from feature list above and assigned to mu, sigma
                    Gal_fit_mu = bestfit_features_list[0]
                    Gal_fit_sigma = bestfit_features_list[1]
                    
                    # Now, we will use method 'eval' to evalute our model function. Here, we will calculate
                    # confidence interval of the fitted variable parameters, determine the uncertainties of our model,
                    # and plot the uncertainties as number of sigma away from the fit parameters
                    
                    # a) Confidence interval: Use F-test method. The optimized/fit parameters are compared to a specific value.
                    # Read: lmfit.github.io/lmfit-py/confidence.html#confidence-chapter
                    # b) Uncertainty: Calculate the uncertainties of the fitted parameters. This means, given a specific value
                    # of sigma away from the fitted parameters, what are the predicted y values. We get to see how far the
                    # predictions are from the actual data, and the fitted curve.
                    ci = Gal_result.conf_interval()
                    
                    
                                       
                    # Compile Peak Features for Gal
                    GaltempmuSV['NormPeakIntensity'] = yGaldata.max()
                    GaltempmuSV['Empirical mu'] = mu_Gal
                    GaltempmuSV['Gal_Gaussian_mu'] = Gal_fit_mu
                    GaltempmuSV['Gal_Gaussian_sigma'] = Gal_fit_sigma
                    GaltempmuSV['Gal_Rsquared'] = r_squared
                    norm_GalPeakFeatures = pd.concat([norm_GalPeakFeatures,GaltempmuSV])
                    norm_GalPeakFeatures.reset_index(drop = True, inplace = True)
                       
                     # Plot fit over original data. We will display original data, initial fit with given initial 
                     # parameters and best fit values. 

                    plt.figure()
                    titlelipid = 'GalCer(d18:1/' + str(keys_dataGal[i]) + ')'
                    plt.ylabel('Normalized Signal Intensity')
                    plt.xlabel('CoV')
                    plt.title((titleGal +' at '+'SV '+str(SV) + ' ' + 'Rep ' + str(duplicate)), fontsize = 14)
                    
                    plt.plot(xGaldata, yGaldata, 'o', color='grey', label='Original Gal')
                    plt.plot(xGaldata, Gal_result.init_fit, '--', color='darkorange', label='Initial fit')
                    plt.plot(xGaldata, Gal_result.best_fit, '-', color='green', label='Best fit')
                    plt.plot(xGaldata, y_Gal_modified1, '--', label='Less Optimal Fit 1', color = 'goldenrod')
                    plt.plot(xGaldata, y_Gal_modified2, '--', label='Less Optimal Fit 2', color = 'gold')
                    plt.legend()
                    temp_Gal = plt.gcf()
                    plt.show()
                    plt.draw()
                    plotname = isomer + keys_dataGal[i] + '_SV' + str(SV) + '_rep' + str(duplicate)
                    plotname = plotname.replace(":","-")
                    plotpath = os.path.join(currentpath, M1_Gal_GaussPlots, plotname)
                    temp_Gal.savefig(plotpath)
                    
                  
                    
    elif isomer == "Glc":
        for i in range(len(list_Lipids)):
            Glctemp = (norm_rawGlc[norm_rawGlc['Lipid Species']==keys_dataGlc[i]]).dropna()
            Glctempmu = raw_GlcPeakCoV[raw_GlcPeakCoV['Lipid Species']==keys_dataGlc[i]]
            
            minSV_Glc = Glctemp['SV'].min()
            maxSV_Glc = Glctemp['SV'].max()
            stepSV = int((maxSV_Glc - minSV_Glc) / ((Glctemp['SV'].nunique())-1))
            
            for duplicate in range(1, nRep+1):
                for SV in range(minSV_Glc, maxSV_Glc+stepSV, stepSV):
                    GlctempSV = Glctemp[(Glctemp['SV']==SV)&(Glctemp['replicateNum']==duplicate)]
                    GlctempmuSV = Glctempmu[(Glctempmu['SV']==SV) & (Glctempmu['replicateNum']==duplicate)]
                    
                    xGlcdata = ar(GlctempSV["COV"])
                    yGlcdata = ar(GlctempSV["NormPeakIntensity"])
                    mu_Glc = float(sum(GlctempmuSV['COV'])/(GlctempmuSV.shape[0])) #this will be the start point
                    
                    # Use Model fit to fit data. Here we use Lmfit to build a Model wrapping the gaussian model function
                    # We assign initial values for this model with mu as the average mu across all CoV for a specific SV (peakCoV)
                    # and sigma at 1.
                    # As well, here, we use method 'fit' of the model to fit data to this model with the initial parameters
                    gmodel = Model(gaussian)
                    Glc_result = gmodel.fit(yGlcdata, x=xGlcdata, mu = mu_Glc, sigma = 1)
                    
                    
                    # Modified parameters set 1 for less optimal fit_FOR GLC
                    modified_Glc_params1 = Glc_result.init_params.copy()
                    modified_Glc_params1['mu'].value = mu_Glc
                    modified_Glc_params1['sigma'].value = 0.75
                    
                    # Calculate new model 1 with modified parameters set 1_FOR GLC
                    y_Glc_modified1 = gmodel.eval(modified_Glc_params1, x=xGlcdata)
                    
                    # Modified parameters set 2 for less optimal fit_FOR GLC
                    modified_Glc_params2 = Glc_result.init_params.copy()
                    modified_Glc_params2['mu'].value = mu_Glc
                    modified_Glc_params2['sigma'].value = 0.25
                    
                    # Calculate new model 2 with modified parameters set 2_FOR GLC
                    y_Glc_modified2 = gmodel.eval(modified_Glc_params2, x=xGlcdata)
                    
                    # The ModelResult methods provide fit parameters and results from the model. We use 'summary' method
                    # to output the statistics and attributes of the model's results. This is a dictionary.
                    # We then derive the value for key "best_values" from the summary, which are mu and sigma
                    gmodel_report = Glc_result.fit_report()
                    gmodel_summary = Glc_result.summary()
                    bestfit_features = gmodel_summary["best_values"]
                    bestfit_features_list = list(bestfit_features.values())
                    r_squared = gmodel_summary["rsquared"]
                    
                    
                    # The best fit values are now derived from feature list above and assigned to mu, sigma
                    Glc_fit_mu = bestfit_features_list[0]
                    Glc_fit_sigma = bestfit_features_list[1]
                    
                    # Now, we will use method 'eval' to evalute our model function. Here, we will calculate
                    # confidence interval of the fitted variable parameters, determine the uncertainties of our model,
                    # and plot the uncertainties as number of sigma away from the fit parameters
                    
                    # a) Confidence interval: Use F-test method. The optimized/fit parameters are compared to a specific value.
                    # Read: lmfit.github.io/lmfit-py/confidence.html#confidence-chapter
                    # b) Uncertainty: Calculate the uncertainties of the fitted parameters. This means, given a specific value
                    # of sigma away from the fitted parameters, what are the predicted y values. We get to see how far the
                    # predictions are from the actual data, and the fitted curve.
                    ci = Glc_result.conf_interval()
                    Glc_uncertain = Glc_result.eval_uncertainty(sigma = 3)
                                       
                                       
                    # Compile Peak Features for Glc
                    GlctempmuSV['NormPeakIntensity'] = yGlcdata.max()
                    GlctempmuSV['Empirical mu'] = mu_Glc
                    GlctempmuSV['Glc_Gaussian_mu'] = Glc_fit_mu
                    GlctempmuSV['Glc_Gaussian_sigma'] = Glc_fit_sigma
                    GlctempmuSV['Glc_Rsquared'] = r_squared
                    norm_GlcPeakFeatures = pd.concat([norm_GlcPeakFeatures,GlctempmuSV])
                    norm_GlcPeakFeatures.reset_index(drop = True, inplace = True)
                       
                    # Plot fit over original data. We will display original data, initial fit with given initial 
                    # parameters and best fit values. 
                     
                    plt.figure()
                    titlelipid = 'GlcCer(d18:1/' + str(keys_dataGlc[i]) + ')'
                    plt.ylabel('Normalized Scanned Intensity')
                    plt.xlabel('CoV')
                    plt.title((titleGlc +' at '+'SV '+str(SV) + ' ' + 'Rep ' + str(duplicate)), fontsize = 14)
                    plt.plot(xGlcdata, yGlcdata, 'o', color='grey', label='Original Glc')
                    plt.plot(xGlcdata, Glc_result.init_fit, '--', color='darkorange', label='Initial fit')
                    plt.plot(xGlcdata, Glc_result.best_fit, '-', color='blue', label='Best fit')
                    plt.plot(xGlcdata, y_Glc_modified1, '--', label='Less Optimal Fit 1', color = 'goldenrod')
                    plt.plot(xGlcdata, y_Glc_modified2, '--', label='Less Optimal Fit 2', color = 'gold')
                    plt.legend()
                    temp_Glc = plt.gcf()
                    plt.show()
                    plt.draw()
                    plotname = isomer + keys_dataGlc[i] + '_SV' + str(SV) + '_rep' + str(duplicate)
                    plotname = plotname.replace(":","-")
                    plotpath = os.path.join(currentpath, M1_Glc_GaussPlots, plotname)
                    temp_Glc.savefig(plotpath)
plt.close()    

## Output Gaussian files

norm_GalPeakFeatures.to_excel('Module1_Outputs/norm_GalPeakFeatures.xlsx')
norm_GlcPeakFeatures.to_excel('Module1_Outputs/norm_GlcPeakFeatures.xlsx')


In [ ]:
# ii. Averaged Gaussian features extraction from all replicates

Galfeatures = pd.DataFrame()
Galfeatures_combined = pd.DataFrame()
Glcfeatures = pd.DataFrame()
Glcfeatures_combined = pd.DataFrame()


for i in range(len(list_Lipids)):
    Galfeatures_temp = (norm_GalPeakFeatures[norm_GalPeakFeatures['Lipid Species']==keys_dataGal[i]])
    Glcfeatures_temp = (norm_GlcPeakFeatures[norm_GlcPeakFeatures['Lipid Species']==keys_dataGlc[i]])
    
    minSV = Galfeatures_temp['SV'].min()
    maxSV = Galfeatures_temp['SV'].max()
    stepSV = int((maxSV - minSV) / ((Galfeatures_temp['SV'].nunique())-1))
     
    
    for SV in range(minSV, maxSV+stepSV, stepSV):
        GalfeaturesSV_temp = Galfeatures_temp[Galfeatures_temp['SV']==SV]
        GlcfeaturesSV_temp = Glcfeatures_temp[Glcfeatures_temp['SV']==SV]
        
        cols = ["Lipid Species", "SV", "chain length", "degree of unsaturation"]
        Galfeatures = GalfeaturesSV_temp[cols]
        Galfeatures.drop_duplicates(inplace = True)
        
        Glcfeatures = GlcfeaturesSV_temp[cols]
        Glcfeatures.drop_duplicates(inplace = True)
        
        
        Galfeatures['mu_Gal'] =  GalfeaturesSV_temp.loc[:,'Gal_Gaussian_mu'].mean()
        Galfeatures['sigma_Gal'] = GalfeaturesSV_temp.loc[:,'Gal_Gaussian_sigma'].mean()
        Galfeatures['peakIntensity_at_mu_Gal'] = GalfeaturesSV_temp.loc[:,'Intensity'].mean()
        
        
        Glcfeatures['mu_Glc'] =  GlcfeaturesSV_temp.loc[:,'Glc_Gaussian_mu'].mean()
        Glcfeatures['sigma_Glc'] = GlcfeaturesSV_temp.loc[:,'Glc_Gaussian_sigma'].mean()
        Glcfeatures['peakIntensity_at_mu_Glc'] = GlcfeaturesSV_temp.loc[:,'Intensity'].mean()
        
                
        Galfeatures_combined = pd.concat([Galfeatures_combined, Galfeatures])
        Glcfeatures_combined = pd.concat([Glcfeatures_combined, Glcfeatures])
        
        if Galfeatures_combined.shape[0] > Glcfeatures_combined.shape[0]:
            CombinedAnalysis = pd.merge(Galfeatures_combined, Glcfeatures_combined, how = "outer", on = ["Lipid Species", "SV", "chain length", "degree of unsaturation"])
            
        else:
            CombinedAnalysis = pd.merge(Glcfeatures_combined, Galfeatures_combined, how = "outer", on = ["Lipid Species", "SV", "chain length", "degree of unsaturation"])
            
        CombinedAnalysis.reset_index(drop = True, inplace = True)
        CombinedAnalysis.drop(CombinedAnalysis.columns[CombinedAnalysis.columns.str.contains('unnamed', case=False)], axis = 1, inplace = True)
              
## Output Gaussian files
CombinedAnalysis.to_excel('Module1_Outputs/CombinedAnalysis.xlsx')

## Calculate average sigma across all lipids, at each SV

meanAllsigma = CombinedAnalysis.groupby('SV').mean('sigma_Glc', 'sigma_Gal').reset_index()
meanAllsigma.to_excel('Module1_Outputs/meanAllsigma.xlsx', columns =['SV','sigma_Glc', 'sigma_Gal'])

## STEP 5. Find the intersection point of Glc and Gal

In [ ]:
Intersected_gauss = pd.DataFrame()

def solve(mu_Gal,mu_Glc,sig_Gal,sig_Glc): 
  a = 1/(2*sig_Gal**2) - 1/(2*sig_Glc**2)
  b = mu_Glc/(sig_Glc**2) - mu_Gal/(sig_Gal**2)
  c = mu_Gal**2 /(2*sig_Gal**2) - mu_Glc**2 / (2*sig_Glc**2) - np.log(sig_Glc/sig_Gal)
  return np.roots([a,b,c])

for i in range(len(list_Lipids)):
    intersect_temp = (CombinedAnalysis[CombinedAnalysis['Lipid Species']==keys_dataGal[i]])
    
    minSV = intersect_temp['SV'].min()
    maxSV = intersect_temp['SV'].max()
    stepSV = int((maxSV - minSV) / ((intersect_temp['SV'].nunique())-1))
    
       
    for SV in range(minSV, maxSV+stepSV, stepSV):
        intersectSV_temp = intersect_temp[intersect_temp['SV']==SV]
        
        if intersectSV_temp["mu_Gal"].notna() is False:
            continue
        else:
            mu_Gal = float(intersectSV_temp["mu_Gal"])
            sig_Gal = float(intersectSV_temp["sigma_Gal"])
        
        if intersectSV_temp["mu_Glc"].notna() is False:
            continue
        else:
            mu_Glc = float(intersectSV_temp["mu_Glc"])
            sig_Glc = float(intersectSV_temp["sigma_Glc"])
        
        gauss_features = [mu_Gal, mu_Glc, sig_Gal, sig_Glc] 
     
        if any(pd.isna(gauss_features)) is True:
            continue
        else:
            resultCombined = solve(mu_Gal, mu_Glc, sig_Gal, sig_Glc)
        
        
        # Based on peak CoV, this step we predict the intensity by refitting X back to the gaussian function.
        x = np.linspace(-10,10,10000) 
        y_Gal = gaussian(x, mu_Gal, sig_Gal)
        y_Glc = gaussian(x, mu_Glc, sig_Glc)
        
        # Intersection point is limited to only that lies between mu of Glc and Gal
        mu_range_max = max(mu_Glc, mu_Gal)
        mu_range_min = min(mu_Glc, mu_Gal)
        
        intersect = max([point for point in resultCombined if mu_range_min < point < mu_range_max], default = "NAN")
        
        if type(intersect) is str:
            y_at_intersect = intersect
            
        else:
            #y_at_intersect = gaussian(intersect, mu_Gal, sig_Gal) # Result is different if Gal gaussian features are choosen to determine the intersected point.
            y_at_intersect = gaussian(intersect, mu_Glc, sig_Glc) # We choose Glc here because it is the left limit.
         
        if intersect != "NAN" and (y_at_intersect < 0.5 ):              
            intersectSV_temp["Valley CoV"] = intersect # Valley is defined as separation

        intersectSV_temp["Intersection point"] = intersect
        intersectSV_temp["Norm Intensity at Intersection"] = y_at_intersect
        
        ax = []
        ax = plt.gca()
        ax.set_xlim([minCoV, maxCoV])

        
        titlelipid = 'Gaussian distribution plot based on averaged mu and sigma of \n GlcCer(d18:1/' + str(keys_dataGal[i]) + ' and GalCer(d18:1/' + str(keys_dataGal[i]) + ') at SV ' + str(SV)
        plt.ylabel('Normalized Signal Intensity')
        plt.xlabel('CoV')
        plt.title(titlelipid)
        plot1 = plt.plot(x, y_Gal, color='green', label='GalCer(d18:1/' + str(keys_dataGal[i]))
        plot2 = plt.plot(x, y_Glc, color='blue', label='GlcCer(d18:1/' + str(keys_dataGlc[i]))
        
        if intersect != "NAN":
          plot3 = plt.plot(intersect, y_at_intersect, 'ro', fillstyle='none', label = 'Intersection point')
          
        plt.legend(loc='upper right')
        temp_plot = plt.gcf()
        plt.show()
        plt.draw()
        
        plotname = 'GaussIntersected Glc' + keys_dataGal[i] + '_Gal' + keys_dataGlc[i] + '_SV' + str(SV) + '_rep' + str(duplicate)
        plotname = plotname.replace(":","-")
        plotpath = os.path.join(currentpath, M1_Intersect_GaussPlots, plotname)
        temp_plot.savefig(plotpath)
       
        Intersected_gauss = pd.concat([Intersected_gauss, intersectSV_temp])
        Intersected_gauss.reset_index(drop = True, inplace = True)
        
# Binary Separation Column

Intersected_gauss['Sep or InSep'] = np.where(Intersected_gauss['Valley CoV'].isnull(), "Inseparable", "Separable")
        
# Output intersection point to files

Intersected_gauss = Intersected_gauss.replace('NAN','')
Intersected_gauss.to_excel('Module1_Outputs/Intersected gauss.xlsx')

#Intersected_gauss.to_excel('Intersected gauss_Glc.xlsx')

# By this point, we have had all the features and data necessary to start performing 1) Training of a model, 2) Testing each model,
# and 3) Pick a Model for prediction. We will write this in another set of script.

In [ ]:
print("Please move on to the next module, either Module 2 or Module 3.")